CrewAI - Gemini 01
https://youtu.be/B8s-FyN4UeE?si=JF6BfX1aOa8tdxl3

CrewAI 是一個自治代理框架，使您能夠建立一組代理來完成複雜的任務。 它在功能上與 AutoGen 非常相似。 您還將學習如何將 Google 的免費 G​​emini API 與 CrewAI 結合使用。

影片是用 Google Colab 來完成的。所以，同時用本地的 jupyter-lab 跟 Google Colab 同時來操作看看囉

In [ ]:
# 安裝crewai, duckduckgo-search, langchain-google-genai
!pip install crewai duckduckgo-search langchain-google-genai

In [15]:
# 匯入套件，用不到的先#取消看看
import os
from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_community.llms import OpenAI, Ollama
# from langchain_openai import ChatOpenAI
from crewai import Agent, Task, Crew, Process

# 匯入工具
from tools.browser_tools import BrowserTools
from tools.calculator_tools import CalculatorTools
from tools.file_tools import FileTools
from tools.search_tools import SearchTools
from tools.sec_tools import SECTools
from tools.template_tools import TemplateTools
# from langchain.tools.yahoo_finance_news import YahooFinanceNewsTool

設定 LLM

In [16]:
# gpt35 = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7),
# gpt4 = ChatOpenAI(model_name="gpt-4", temperature=0.7),
# ollama = Ollama(model="openhermes"),
# gemini 有時候是甚麼原因會報錯，不確定，但目前可以使用
# gemini = ChatGoogleGenerativeAI(model = "gemini-pro", verbose = True, temperature = 0.6, google_api_key = os.environ["GOOGLE_API_KEY"]),
gemini = ChatGoogleGenerativeAI(model="gemini-pro",
                             verbose = True,
                             temperature = 0.6,
                             #google_api_key="<Enter Google Gemini API KEY>"
                             )

設定 tools

In [17]:
from langchain.tools import DuckDuckGoSearchRun
search_tool = DuckDuckGoSearchRun()

tools=[
    search_tool,
    BrowserTools.scrape_and_summarize_website,
    CalculatorTools.calculate,
    FileTools.write_file,     
    SearchTools.search_internet,
    SearchTools.search_news,
    SECTools.search_10q,
    SECTools.search_10k,
    TemplateTools.learn_landing_page_options,
    TemplateTools.copy_landing_page_template_to_project_folder,
    # YahooFinanceNewsTool()
    ]


接下來開始定義 Agent，這部分取決於你想要用多少個 Agent，每個 Agent 都有不同的 role(角色)，不同的 goal(目標)及不同的 backgstory(背景故事)，或者不同的 system prompt(系統提示詞)。

In [18]:
researcher = Agent(
    role = 'Senior Research Analyst',
    goal = 'Uncover cutting-edge development in AI and data science',
    backstory = """ You work in a leading tech think tank.
    Your expertise lies in identifying emerging trends.
    You have a knack for dissecting complex data and presenting actionable insights.""",
    verbose=True,
    allow_delegation=False,
    llm = gemini,
    tools = [search_tool]
    
)
writer = Agent(
    role='Tech Content Strategist',
    goal='Craft compelling content on tech advancements',
    backstory="""You are a renowned Content Strategist, known for
    your insightful and engaging articles.
    You transform complex concepts into compelling narratives. And you are expert at writing in traditional chinese""",
    verbose=True,
    allow_delegation=True,
    llm = gemini,
)

接下來提供這些 Agent 要執行的 task(任務列表) 

In [19]:
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report""",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.
  Your final answer MUST be the full blog post of at least 5 paragraphs.
  And always use Traditional chinese to express.
  """,
  agent=writer
)

使用順序過程實例化您的 Agent 成員(crew)

In [20]:
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  verbose=2, # You can set it to 1 or 2 to different logging levels
)

讓你的 Agent成員(crew) 開始工作！並輸出成果！

In [22]:
result = crew.kickoff()

print("######################")
print(result)

[DEBUG]: Working Agent: Senior Research Analyst
[INFO]: Starting Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.
  Your final answer MUST be a full analysis report


> Entering new CrewAgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: duckduckgo_search
Action Input: What are the key trends in AI in 2024?Tool duckduckgo_search has been used.
 

[1] AI will have an estimated 21% net increase on the United States GDP by 2030 AI is expected to contribute a significant 21% net increase to the United States GDP by 2030, showcasing its... Five Key Trends in AI and Data Science for 2024 These developing issues should be on every leader's radar screen, data executives say. Thomas H. Davenport and Randy Bean January 09, 2024 Reading Time: 7 min Carolyn Geason-Beissel/MIT SMR | Getty Images Artificial intelligence and data science became front-page news in 2023. 20

d:\TOMO.Project\crewAI\venv\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")
